In [1]:
#Run Once
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.1.1/spark-3.1.1-bin-hadoop2.7.tgz
!tar xf spark-3.1.1-bin-hadoop2.7.tgz
!pip install -q findspark
#Run Once
import os
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop2.7"
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()
spark

In [4]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels

--2021-04-19 10:52:08--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 3.229.173.44, 54.226.184.31, 18.205.129.29, ...
Connecting to bin.equinox.io (bin.equinox.io)|3.229.173.44|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14746350 (14M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.1’

ngrok-stable-linux- 100%[===================>]  14.06M  9.13MB/s    in 1.5s    

2021-04-19 10:52:10 (9.13 MB/s) - ‘ngrok-stable-linux-amd64.zip.1’ saved [14746350/14746350]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ngrok                   
{"tunnels":[{"name":"command_line","uri":"/api/tunnels/command_line","public_url":"https://7f8d9cc362ea.ngrok.io","proto":"https","config":{"addr":"http://localhost:4050","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0

In [6]:
import pandas as pd
from pyspark.sql.functions import to_timestamp
from pyspark.sql.types import *
from pyspark.sql import functions as F
sc = spark.sparkContext
from pyspark.sql import Row
from pyspark.sql.functions import isnull, when, count, col
from pyspark.sql.functions import udf

In [7]:
Final_X_Ydataset = spark.read.csv('/content/drive/MyDrive/Colab datasets/Final/Final_X_Y_3Hdataset_v3.csv', header = True, inferSchema=True)
Final_X_Ydataset.printSchema()

root
 |-- date: string (nullable = true)
 |-- rain_OAK_PARK: integer (nullable = true)
 |-- temp_OAK_PARK: double (nullable = true)
 |-- rhum_OAK_PARK: integer (nullable = true)
 |-- msl_OAK_PARK: double (nullable = true)
 |-- wdsp_OAK_PARK: integer (nullable = true)
 |-- wddir_OAK_PARK: integer (nullable = true)
 |-- rain_SHANNON_AIRPORT: integer (nullable = true)
 |-- temp_SHANNON_AIRPORT: double (nullable = true)
 |-- rhum_SHANNON_AIRPORT: integer (nullable = true)
 |-- msl_SHANNON_AIRPORT: double (nullable = true)
 |-- wdsp_SHANNON_AIRPORT: integer (nullable = true)
 |-- wddir_SHANNON_AIRPORT: integer (nullable = true)
 |-- rain_ATHENRY: integer (nullable = true)
 |-- temp_ATHENRY: double (nullable = true)
 |-- rhum_ATHENRY: integer (nullable = true)
 |-- msl_ATHENRY: double (nullable = true)
 |-- wdsp_ATHENRY: integer (nullable = true)
 |-- wddir_ATHENRY: integer (nullable = true)
 |-- rain_BELMULLET: integer (nullable = true)
 |-- temp_BELMULLET: double (nullable = true)
 |-- rhu

In [8]:
Final_X_Ydataset.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
date,87670,None,None,01/01/2011 00:00,31/12/2020 21:00
rain_OAK_PARK,87670,0.2795483061480552,0.7190902329547432,0,3
temp_OAK_PARK,87670,10.125015398654057,5.177328962183774,-5.6,29.5
rhum_OAK_PARK,87670,82.761161172579,12.584318702595978,20,100
...,...,...,...,...,...
wddir_Gurteen,87670,200.84772442112467,79.66354074464343,0,360
rain_sligo_Y3H,87670,0.4521501083608988,0.8657423078024291,0,3
temp_sligo_Y3H,87670,9.464827192882412,4.913974583831496,-8.4,29.4
rhum_sligo_Y3H,87670,83.86566670468804,11.95277761570073,0,100


In [10]:
## See if we have missing values
from pyspark.sql.functions import isnull, when, count, col

Final_X_Ydataset.select([count(when(isnull(c), c)).alias(c) for c in Final_X_Ydataset.columns]).show()

+----+-------------+-------------+-------------+------------+-------------+--------------+--------------------+--------------------+--------------------+-------------------+--------------------+---------------------+------------+------------+------------+-----------+------------+-------------+--------------+--------------+--------------+-------------+--------------+---------------+-------------+-------------+-------------+------------+-------------+--------------+----------------+----------------+----------------+---------------+----------------+-----------------+-----------------+-----------------+-----------------+----------------+-----------------+------------------+-------------------+-------------------+-------------------+------------------+-------------------+--------------------+-----------+-----------+-----------+----------+-----------+------------+--------------+--------------+--------------+-------------+--------------+---------------+------------------+------------------+--

In [11]:
Final_X_Ydataset = Final_X_Ydataset.na.fill(value=0)

In [12]:
Final_X_Ydataset.select([count(when(isnull(c), c)).alias(c) for c in Final_X_Ydataset.columns]).show()

+----+-------------+-------------+-------------+------------+-------------+--------------+--------------------+--------------------+--------------------+-------------------+--------------------+---------------------+------------+------------+------------+-----------+------------+-------------+--------------+--------------+--------------+-------------+--------------+---------------+-------------+-------------+-------------+------------+-------------+--------------+----------------+----------------+----------------+---------------+----------------+-----------------+-----------------+-----------------+-----------------+----------------+-----------------+------------------+-------------------+-------------------+-------------------+------------------+-------------------+--------------------+-----------+-----------+-----------+----------+-----------+------------+--------------+--------------+--------------+-------------+--------------+---------------+------------------+------------------+--

In [13]:
Final_X_Ydataset.columns

['date',
 'rain_OAK_PARK',
 'temp_OAK_PARK',
 'rhum_OAK_PARK',
 'msl_OAK_PARK',
 'wdsp_OAK_PARK',
 'wddir_OAK_PARK',
 'rain_SHANNON_AIRPORT',
 'temp_SHANNON_AIRPORT',
 'rhum_SHANNON_AIRPORT',
 'msl_SHANNON_AIRPORT',
 'wdsp_SHANNON_AIRPORT',
 'wddir_SHANNON_AIRPORT',
 'rain_ATHENRY',
 'temp_ATHENRY',
 'rhum_ATHENRY',
 'msl_ATHENRY',
 'wdsp_ATHENRY',
 'wddir_ATHENRY',
 'rain_BELMULLET',
 'temp_BELMULLET',
 'rhum_BELMULLET',
 'msl_BELMULLET',
 'wdsp_BELMULLET',
 'wddir_BELMULLET',
 'rain_CASEMENT',
 'temp_CASEMENT',
 'rhum_CASEMENT',
 'msl_CASEMENT',
 'wdsp_CASEMENT',
 'wddir_CASEMENT',
 'rain_CLAREMORRIS',
 'temp_CLAREMORRIS',
 'rhum_CLAREMORRIS',
 'msl_CLAREMORRIS',
 'wdsp_CLAREMORRIS',
 'wddir_CLAREMORRIS',
 'rain_CORK_AIRPORT',
 'temp_CORK_AIRPORT',
 'rhum_CORK_AIRPORT',
 'msl_CORK_AIRPORT',
 'wdsp_CORK_AIRPORT',
 'wddir_CORK_AIRPORT',
 'rain_DUBLIN_AIRPORT',
 'temp_DUBLIN_AIRPORT',
 'rhum_DUBLIN_AIRPORT',
 'msl_DUBLIN_AIRPORT',
 'wdsp_DUBLIN_AIRPORT',
 'wddir_DUBLIN_AIRPORT',
 'rain_

In [14]:
rain = Final_X_Ydataset.select( 'rain_OAK_PARK',
 'temp_OAK_PARK',
 'rhum_OAK_PARK',
 'msl_OAK_PARK',
 'wdsp_OAK_PARK',
 'wddir_OAK_PARK',
 'rain_SHANNON_AIRPORT',
 'temp_SHANNON_AIRPORT',
 'rhum_SHANNON_AIRPORT',
 'msl_SHANNON_AIRPORT',
 'wdsp_SHANNON_AIRPORT',
 'wddir_SHANNON_AIRPORT',
 'rain_DUBLIN_AIRPORT',
 'temp_DUBLIN_AIRPORT',
 'rhum_DUBLIN_AIRPORT',
 'msl_DUBLIN_AIRPORT',
 'wdsp_DUBLIN_AIRPORT',
 'wddir_DUBLIN_AIRPORT',
 'rain_MOORE_PARK',
 'temp_MOORE_PARK',
 'rhum_MOORE_PARK',
 'msl_MOORE_PARK',
 'wdsp_MOORE_PARK',
 'wddir_MOORE_PARK',
 'rain_Sherk_inIsland',
 'temp_Sherk_inIsland',
 'rhum_Sherk_inIsland',
 'msl_Sherk_inIsland',
 'wdsp_Sherk_inIsland',
 'wddir_Sherk_inIsland',
 'rain_MULLINGAR',
 'temp_MULLINGAR',
 'rhum_MULLINGAR',
 'msl_MULLINGAR',
 'wdsp_MULLINGAR',
 'wddir_MULLINGAR',
 'rain_MALIN_HEAD',
 'temp_MALIN_HEAD',
 'rhum_MALIN_HEAD',
 'msl_MALIN_HEAD',
 'wdsp_MALIN_HEAD',
 'wddir_MALIN_HEAD',
 'rain_JOHNSTOWN',
 'temp_JOHNSTOWN',
 'rhum_JOHNSTOWN',
 'msl_JOHNSTOWN',
 'wdsp_JOHNSTOWN',
 'wddir_JOHNSTOWN',
 'rain_MT_DILLON',
 'temp_MT_DILLON',
 'rhum_MT_DILLON',
 'msl_MT_DILLON',
 'wdsp_MT_DILLON',
 'wddir_MT_DILLON',
 'rain_FINNER',
 'temp_FINNER',
 'rhum_FINNER',
 'msl_FINNER',
 'wdsp_FINNER',
 'wddir_FINNER',
 'rain_CLAREMORRIS',
 'temp_CLAREMORRIS',
 'rhum_CLAREMORRIS',
 'msl_CLAREMORRIS',
 'wdsp_CLAREMORRIS',
 'wddir_CLAREMORRIS',
 'rain_VALENTIA_OBSERVATORY',
 'temp_VALENTIA_OBSERVATORY',
 'rhum_VALENTIA_OBSERVATORY',
 'msl_VALENTIA_OBSERVATORY',
 'wdsp_VALENTIA_OBSERVATORY',
 'wddir_VALENTIA_OBSERVATORY',
 'rain_BELMULLET',
 'temp_BELMULLET',
 'rhum_BELMULLET',
 'msl_BELMULLET',
 'wdsp_BELMULLET',
 'wddir_BELMULLET',
 'rain_CASEMENT',
 'temp_CASEMENT',
 'rhum_CASEMENT',
 'msl_CASEMENT',
 'wdsp_CASEMENT',
 'wddir_CASEMENT',
 'rain_CORK_AIRPORT',
 'temp_CORK_AIRPORT',
 'rhum_CORK_AIRPORT',
 'msl_CORK_AIRPORT',
 'wdsp_CORK_AIRPORT',
 'wddir_CORK_AIRPORT',
 'rain_KNOCK_AIRPORT',
 'temp_KNOCK_AIRPORT',
 'rhum_KNOCK_AIRPORT',
 'msl_KNOCK_AIRPORT',
 'wdsp_KNOCK_AIRPORT',
 'wddir_KNOCK_AIRPORT',
 'rain_Newport',
 'temp_Newport',
 'rhum_Newport',
 'msl_Newport',
 'wdsp_Newport',
 'wddir_Newport',
 'rain_RochesPoint',
 'temp_RochesPoint',
 'rhum_RochesPoint',
 'msl_RochesPoint',
 'wdsp_RochesPoint',
 'wddir_RochesPoint',
 'rain_ATHENRY',
 'temp_ATHENRY',
 'rhum_ATHENRY',
 'msl_ATHENRY',
 'wdsp_ATHENRY',
 'wddir_ATHENRY',
 'rain_Markree',
 'temp_Markree',
 'rhum_Markree',
 'msl_Markree',
 'wdsp_Markree',
 'wddir_Markree',
 'rain_Dunsany',
 'temp_Dunsany',
 'rhum_Dunsany',
 'msl_Dunsany',
 'wdsp_Dunsany',
 'wddir_Dunsany',
 'rain_BALLYHAISE',
 'temp_BALLYHAISE',
 'rhum_BALLYHAISE',
 'msl_BALLYHAISE',
 'wdsp_BALLYHAISE',
 'wddir_BALLYHAISE',
 'rain_Gurteen',
 'temp_Gurteen',
 'rhum_Gurteen',
 'msl_Gurteen',
 'wdsp_Gurteen',
 'wddir_Gurteen',
 'rain_sligo_Y3H')

In [15]:
rain.show(5)

+-------------+-------------+-------------+------------+-------------+--------------+--------------------+--------------------+--------------------+-------------------+--------------------+---------------------+-------------------+-------------------+-------------------+------------------+-------------------+--------------------+---------------+---------------+---------------+--------------+---------------+----------------+-------------------+-------------------+-------------------+------------------+-------------------+--------------------+--------------+--------------+--------------+-------------+--------------+---------------+---------------+---------------+---------------+--------------+---------------+----------------+--------------+--------------+--------------+-------------+--------------+---------------+--------------+--------------+--------------+-------------+--------------+---------------+-----------+-----------+-----------+----------+-----------+------------+---------------

In [16]:
import six
for i in rain.columns:
    if not( isinstance(rain.select(i).take(1)[0][0], six.string_types)):
        print( "Correlation to Pridicted Rain for ", i, rain.stat.corr('rain_sligo_Y3H',i))

Correlation to Pridicted Rain for  rain_OAK_PARK 0.2134686615269707
Correlation to Pridicted Rain for  temp_OAK_PARK -0.028732989456202437
Correlation to Pridicted Rain for  rhum_OAK_PARK 0.0932869639927928
Correlation to Pridicted Rain for  msl_OAK_PARK -0.0815329755357872
Correlation to Pridicted Rain for  wdsp_OAK_PARK 0.2932709683786244
Correlation to Pridicted Rain for  wddir_OAK_PARK -0.02001640905582671
Correlation to Pridicted Rain for  rain_SHANNON_AIRPORT 0.3158020876988414
Correlation to Pridicted Rain for  temp_SHANNON_AIRPORT -0.030029399584529264
Correlation to Pridicted Rain for  rhum_SHANNON_AIRPORT 0.13500629463998626
Correlation to Pridicted Rain for  msl_SHANNON_AIRPORT -0.2677144065448758
Correlation to Pridicted Rain for  wdsp_SHANNON_AIRPORT 0.2882887061650753
Correlation to Pridicted Rain for  wddir_SHANNON_AIRPORT 0.07217383896493246
Correlation to Pridicted Rain for  rain_DUBLIN_AIRPORT 0.16325674484566557
Correlation to Pridicted Rain for  temp_DUBLIN_AIRPORT 

In [17]:
featurename =[]
corList= []

for i in rain.columns:
    if not( isinstance(rain.select(i).take(1)[0][0], six.string_types)):
      fbuild = []
      cbuild = []
      my_cor = rain.stat.corr('rain_sligo_Y3H', i)
      
      featurename.append(i)
      corList.append(my_cor )

      print( "Correlation to Pridicted Rain for ", i, my_cor)

corrtable = [featurename, corList]

Correlation to Pridicted Rain for  rain_OAK_PARK 0.2134686615269707
Correlation to Pridicted Rain for  temp_OAK_PARK -0.028732989456202437
Correlation to Pridicted Rain for  rhum_OAK_PARK 0.0932869639927928
Correlation to Pridicted Rain for  msl_OAK_PARK -0.0815329755357872
Correlation to Pridicted Rain for  wdsp_OAK_PARK 0.2932709683786244
Correlation to Pridicted Rain for  wddir_OAK_PARK -0.02001640905582671
Correlation to Pridicted Rain for  rain_SHANNON_AIRPORT 0.3158020876988414
Correlation to Pridicted Rain for  temp_SHANNON_AIRPORT -0.030029399584529264
Correlation to Pridicted Rain for  rhum_SHANNON_AIRPORT 0.13500629463998626
Correlation to Pridicted Rain for  msl_SHANNON_AIRPORT -0.2677144065448758
Correlation to Pridicted Rain for  wdsp_SHANNON_AIRPORT 0.2882887061650753
Correlation to Pridicted Rain for  wddir_SHANNON_AIRPORT 0.07217383896493246
Correlation to Pridicted Rain for  rain_DUBLIN_AIRPORT 0.16325674484566557
Correlation to Pridicted Rain for  temp_DUBLIN_AIRPORT 

In [18]:
corrtable

[['rain_OAK_PARK',
  'temp_OAK_PARK',
  'rhum_OAK_PARK',
  'msl_OAK_PARK',
  'wdsp_OAK_PARK',
  'wddir_OAK_PARK',
  'rain_SHANNON_AIRPORT',
  'temp_SHANNON_AIRPORT',
  'rhum_SHANNON_AIRPORT',
  'msl_SHANNON_AIRPORT',
  'wdsp_SHANNON_AIRPORT',
  'wddir_SHANNON_AIRPORT',
  'rain_DUBLIN_AIRPORT',
  'temp_DUBLIN_AIRPORT',
  'rhum_DUBLIN_AIRPORT',
  'msl_DUBLIN_AIRPORT',
  'wdsp_DUBLIN_AIRPORT',
  'wddir_DUBLIN_AIRPORT',
  'rain_MOORE_PARK',
  'temp_MOORE_PARK',
  'rhum_MOORE_PARK',
  'msl_MOORE_PARK',
  'wdsp_MOORE_PARK',
  'wddir_MOORE_PARK',
  'rain_Sherk_inIsland',
  'temp_Sherk_inIsland',
  'rhum_Sherk_inIsland',
  'msl_Sherk_inIsland',
  'wdsp_Sherk_inIsland',
  'wddir_Sherk_inIsland',
  'rain_MULLINGAR',
  'temp_MULLINGAR',
  'rhum_MULLINGAR',
  'msl_MULLINGAR',
  'wdsp_MULLINGAR',
  'wddir_MULLINGAR',
  'rain_MALIN_HEAD',
  'temp_MALIN_HEAD',
  'rhum_MALIN_HEAD',
  'msl_MALIN_HEAD',
  'wdsp_MALIN_HEAD',
  'wddir_MALIN_HEAD',
  'rain_JOHNSTOWN',
  'temp_JOHNSTOWN',
  'rhum_JOHNSTOWN'

In [19]:



R = Row("Name", "Score")
myscores = sc.parallelize([R(*r) for r in zip(*corrtable)]).toDF()

In [20]:
myscores.show(5)

+-------------+--------------------+
|         Name|               Score|
+-------------+--------------------+
|rain_OAK_PARK|  0.2134686615269707|
|temp_OAK_PARK|-0.02873298945620...|
|rhum_OAK_PARK|  0.0932869639927928|
| msl_OAK_PARK| -0.0815329755357872|
|wdsp_OAK_PARK|  0.2932709683786244|
+-------------+--------------------+
only showing top 5 rows



In [21]:
from pyspark.sql.functions import col

myscores.sort(col("Score").asc()).show(15, False)
myscores.sort(col("Score").desc()).show(15, False )


+------------------------+--------------------+
|Name                    |Score               |
+------------------------+--------------------+
|msl_MALIN_HEAD          |-0.3127986779389739 |
|msl_BELMULLET           |-0.3054675416614531 |
|msl_KNOCK_AIRPORT       |-0.29832499441023974|
|msl_CLAREMORRIS         |-0.2942897758034423 |
|msl_DUBLIN_AIRPORT      |-0.27007467981522293|
|msl_CASEMENT            |-0.2697638158323241 |
|msl_SHANNON_AIRPORT     |-0.2677144065448758 |
|msl_MOORE_PARK          |-0.24601331288851852|
|msl_CORK_AIRPORT        |-0.24022298987981158|
|msl_MULLINGAR           |-0.2179509181209707 |
|msl_RochesPoint         |-0.18627425452594354|
|msl_Dunsany             |-0.16001496353082956|
|msl_Gurteen             |-0.14942298907175117|
|msl_BALLYHAISE          |-0.12941997023529817|
|msl_VALENTIA_OBSERVATORY|-0.10757164704959833|
+------------------------+--------------------+
only showing top 15 rows

+-------------------------+-------------------+
|Name         

very interesting!

rain features seem to be be the most positively correlated with Pr-rain along with windspeed


and 

msl (Mean Sea-level Pressure) features seems to be most negativily correleated with Pr-rain, then wind-direction and temperature. 

ok time to maker the vector


In [24]:
from pyspark.ml.feature import VectorAssembler
vectorAssembler = VectorAssembler(inputCols = [ 'rain_Newport',
 'rain_CLAREMORRIS',
 'rain_KNOCK_AIRPORT',
 'rain_BELMULLET',
 'rain_Markree',
 'wdsp_VALENTIA_OBSERVATORY',
 'rain_ATHENRY',
 'rain_MT_DILLON',
 'rain_FINNER',
 'rain_SHANNON_AIRPORT',
 'wdsp_RochesPoint',
 'wdsp_Gurteen',
 'wdsp_CASEMENT',
 'wdsp_OAK_PARK',
 'msl_MALIN_HEAD',
  'msl_BELMULLET',
  'msl_KNOCK_AIRPORT',
  'msl_CLAREMORRIS',
  'msl_DUBLIN_AIRPORT', ], outputCol = 'features')
vector_rain_df = vectorAssembler.transform(rain)
vector_rain_df = vector_rain_df.select(['features', 'rain_sligo_Y3H'])
vector_rain_df.show(3, truncate=False)

+---------------------------------------------------------------------------------------------------+--------------+
|features                                                                                           |rain_sligo_Y3H|
+---------------------------------------------------------------------------------------------------+--------------+
|[2.0,3.0,2.0,2.0,2.0,11.0,0.0,2.0,0.0,0.0,8.0,15.0,17.0,8.0,994.5,999.0,999.0,999.1,1001.3]        |0             |
|(19,[5,10,11,12,13,14,15,16,17,18],[2.0,5.0,6.0,12.0,4.0,1006.3,1009.1,1009.8,1010.1,1010.0])      |0             |
|(19,[5,7,10,11,12,13,14,15,16,17,18],[7.0,1.0,18.0,4.0,6.0,6.0,1002.6,1001.0,1002.5,1002.1,1004.4])|0             |
+---------------------------------------------------------------------------------------------------+--------------+
only showing top 3 rows



In [23]:
from pyspark.ml.feature import VectorAssembler
vectorAssembler = VectorAssembler(inputCols = [ 'rain_Newport',
 'rain_CLAREMORRIS',
 'rain_KNOCK_AIRPORT',
 'rain_BELMULLET' ], outputCol = 'features')
vector_rain_df = vectorAssembler.transform(rain)
vector_rain_df = vector_rain_df.select(['features', 'rain_sligo_Y3H'])
vector_rain_df.show(3, truncate=False)

+-----------------+--------------+
|features         |rain_sligo_Y3H|
+-----------------+--------------+
|[2.0,3.0,2.0,2.0]|0             |
|(4,[],[])        |0             |
|(4,[],[])        |0             |
+-----------------+--------------+
only showing top 3 rows



In [25]:
vector_rain_df.show(10, truncate=True)

+--------------------+--------------+
|            features|rain_sligo_Y3H|
+--------------------+--------------+
|[2.0,3.0,2.0,2.0,...|             0|
|(19,[5,10,11,12,1...|             0|
|(19,[5,7,10,11,12...|             0|
|(19,[5,10,11,12,1...|             0|
|(19,[0,5,10,11,12...|             0|
|[2.0,0.0,0.0,2.0,...|             0|
|[0.0,0.0,0.0,3.0,...|             0|
|(19,[5,9,10,11,12...|             0|
|[1.0,0.0,0.0,2.0,...|             2|
|(19,[0,5,10,11,12...|             0|
+--------------------+--------------+
only showing top 10 rows



In [26]:
# Split the data
(training_data, test_data) = vector_rain_df.randomSplit([0.7, 0.3])

In [27]:
print(training_data.count(), test_data.count())

61390 26280


In [28]:
vector_rain_df.groupBy('rain_sligo_Y3H').count().show()

+--------------+-----+
|rain_sligo_Y3H|count|
+--------------+-----+
|             1| 6142|
|             3| 3498|
|             2|11502|
|             0|66528|
+--------------+-----+



In [29]:
no_rain = vector_rain_df.filter("rain_sligo_Y3H==0")
Light_Rain = vector_rain_df.filter("rain_sligo_Y3H==1")
Moderate_rain = vector_rain_df.filter("rain_sligo_Y3H==2")
Heavy_rain = vector_rain_df.filter("rain_sligo_Y3H==3")

In [30]:
no_raindf = spark.createDataFrame(no_rain.head(3498), no_rain.schema)
Light_Raindf = spark.createDataFrame(Light_Rain.head(3498), Light_Rain.schema)
Moderate_raindf = spark.createDataFrame(Moderate_rain.head(3498), Moderate_rain.schema)
Heavy_raindf = spark.createDataFrame(Heavy_rain.head(3498), Heavy_rain.schema)

In [31]:
BalacedDF = no_raindf.union(Light_Raindf)
BalacedDF = BalacedDF.union(Moderate_raindf)
BalacedDF = BalacedDF.union(Heavy_raindf)

In [32]:
BalacedDF.groupBy('rain_sligo_Y3H').count().show()

+--------------+-----+
|rain_sligo_Y3H|count|
+--------------+-----+
|             1| 3498|
|             3| 3498|
|             2| 3498|
|             0| 3498|
+--------------+-----+



In [33]:
# Split the data
(training_data, test_data) = BalacedDF.randomSplit([0.8, 0.2])

In [34]:
training_data.show(5, False)

+---------------------------------------------------------------------------------------------------+--------------+
|features                                                                                           |rain_sligo_Y3H|
+---------------------------------------------------------------------------------------------------+--------------+
|(19,[0,5,10,11,12,13,14,15,16,17,18],[1.0,1.0,5.0,2.0,3.0,3.0,1038.7,1038.4,1039.0,1038.9,1039.0]) |0             |
|(19,[0,5,10,11,12,13,14,15,16,17,18],[1.0,1.0,11.0,6.0,9.0,7.0,1030.5,1034.6,1034.3,1034.4,1032.1])|0             |
|(19,[0,5,10,11,12,13,14,15,16,17,18],[1.0,2.0,6.0,6.0,8.0,5.0,1028.7,1031.7,1032.9,1032.8,1032.6]) |0             |
|(19,[0,5,10,11,12,13,14,15,16,17,18],[1.0,3.0,7.0,4.0,8.0,2.0,1027.3,1028.7,1029.7,1029.3,1029.5]) |0             |
|(19,[0,5,10,11,12,13,14,15,16,17,18],[1.0,4.0,2.0,5.0,6.0,4.0,1010.5,1010.5,1011.5,1011.4,1012.3]) |0             |
+---------------------------------------------------------------

In [35]:
training_data.groupBy('rain_sligo_Y3H').count().show()

+--------------+-----+
|rain_sligo_Y3H|count|
+--------------+-----+
|             1| 2788|
|             3| 2795|
|             2| 2838|
|             0| 2765|
+--------------+-----+



In [36]:
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
nb = NaiveBayes(smoothing=1.0, modelType="multinomial", labelCol="rain_sligo_Y3H", predictionCol="features1" )
model = nb.fit(training_data)
predictions = model.transform(test_data)
predictions.show(20)

+--------------------+--------------+--------------------+--------------------+---------+
|            features|rain_sligo_Y3H|       rawPrediction|         probability|features1|
+--------------------+--------------+--------------------+--------------------+---------+
|(19,[0,5,10,11,12...|             0|[-8486.1922360986...|[0.99999309518978...|      0.0|
|(19,[0,5,10,11,12...|             0|[-8302.9578010843...|[0.99996056055750...|      0.0|
|(19,[0,5,10,11,12...|             0|[-8441.5486215955...|[0.99996926299344...|      0.0|
|(19,[0,5,10,11,12...|             0|[-8601.6449109270...|[0.99842666003097...|      0.0|
|(19,[0,5,10,11,12...|             0|[-8473.8685279438...|[0.99996020958487...|      0.0|
|(19,[0,5,10,11,12...|             0|[-8457.1583229328...|[0.99819560094809...|      0.0|
|(19,[0,5,10,11,12...|             0|[-8498.2888902177...|[0.85861238908793...|      0.0|
|(19,[0,5,10,11,12...|             0|[-8676.2500111159...|[0.81109442684494...|      0.0|
|(19,[0,5,

In [37]:
evaluator = MulticlassClassificationEvaluator(labelCol="rain_sligo_Y3H", predictionCol="features1",
                                              metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test set accuracy = " + str(accuracy))

Test set accuracy = 0.43300071275837493


In [38]:
# Define the model
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier(labelCol='rain_sligo_Y3H', 
                            featuresCol='features',
                            numTrees =5,
                            maxDepth=5)

In [39]:
# Fit the model
model = rf.fit(training_data)

In [40]:
# Predict with the test dataset
rf_predictions = model.transform(test_data)

In [41]:
rf_predictions.show()

+--------------------+--------------+--------------------+--------------------+----------+
|            features|rain_sligo_Y3H|       rawPrediction|         probability|prediction|
+--------------------+--------------+--------------------+--------------------+----------+
|(19,[0,5,10,11,12...|             0|[1.76863099970865...|[0.35372619994173...|       0.0|
|(19,[0,5,10,11,12...|             0|[1.62195778816098...|[0.32439155763219...|       0.0|
|(19,[0,5,10,11,12...|             0|[1.76863099970865...|[0.35372619994173...|       0.0|
|(19,[0,5,10,11,12...|             0|[1.76863099970865...|[0.35372619994173...|       0.0|
|(19,[0,5,10,11,12...|             0|[1.76863099970865...|[0.35372619994173...|       0.0|
|(19,[0,5,10,11,12...|             0|[1.44882551310768...|[0.28976510262153...|       0.0|
|(19,[0,5,10,11,12...|             0|[1.41231100618959...|[0.28246220123791...|       0.0|
|(19,[0,5,10,11,12...|             0|[1.37129006509640...|[0.27425801301928...|       1.0|

In [42]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

multi_evaluator = MulticlassClassificationEvaluator(labelCol = 'rain_sligo_Y3H', metricName = 'accuracy')
print('Random Forest classifier Accuracy:', multi_evaluator.evaluate(rf_predictions))

Random Forest classifier Accuracy: 0.45652173913043476


decision tree

In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier

dt = DecisionTreeClassifier(featuresCol = 'features', labelCol = 'rain_sligo_Y3H', maxDepth = 3)
dtModel = dt.fit(training_data)
dt_predictions = dtModel.transform(test_data)

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

multi_evaluator = MulticlassClassificationEvaluator(labelCol = 'rain_sligo_Y3H', metricName = 'accuracy')
print('Decision Tree Accuracy:', multi_evaluator.evaluate(dt_predictions))

Decision Tree Accuracy: 0.41663691538736164


In [ ]:
dtModel.featureImportances

SparseVector(19, {0: 0.4862, 1: 0.0604, 3: 0.1368, 4: 0.1135, 14: 0.2031})

**### Logistic Regression Model**

In [ ]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(featuresCol = 'features', labelCol = 'rain_sligo_Y3H', maxIter=20)
lrModel = lr.fit(training_data)
lr_predictions = dtModel.transform(test_data)

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

multi_evaluator = MulticlassClassificationEvaluator(labelCol = 'rain_sligo_Y3H', metricName = 'accuracy')
print('Logistic Regression Accuracy:', multi_evaluator.evaluate(lr_predictions))

Logistic Regression Accuracy: 0.41663691538736164


In [ ]:
evaluator = RegressionEvaluator(
    labelCol="rain_sligo_Y3H", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

IllegalArgumentException: ignored

In [ ]:
lrModel.featureImportances

AttributeError: ignored